<div align="center"><h3>HW4</h3></div>
<div align="center"><h5>Mohammadreza Ghofrani, 400131076</h5></div>

In [1]:
import os
# To disable gpu warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.inception_v3 import InceptionV3

Loading dataset

In [3]:
def load_dataset(split):
    ds = tfds.load(name="beans", split=split, batch_size=-1) 
    ds_ = tfds.as_numpy(ds)
    xds, yds = ds_["image"], ds_["label"]

    return xds, yds

xtrain, ytrain = load_dataset('train')
xval, yval = load_dataset('validation')
xtest, ytest = load_dataset('test')

In [4]:
def reduce_size_to_32by32(xdataset, ydataset):
    n_data, n_class = len(xdataset), 3
    xoutput = np.zeros((n_data, 32, 32))
    youtput = np.zeros((n_data, n_class))
    for i, img_label in enumerate(zip(xdataset, ydataset)):
        img, label = img_label[0], img_label[1]
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        xoutput[i] = cv2.resize(img_gray, (32,32))
        youtput[i][label] += 1
    return tf.expand_dims(xoutput, axis=-1), youtput

xtrain32by32, ytrain32by32 = reduce_size_to_32by32(xtrain, ytrain)
xval32by32, yval32by32 = reduce_size_to_32by32(xval, yval)
xtest32by32, ytest32by32 = reduce_size_to_32by32(xtest, ytest)

# Question 2

LeNet neural network base implementation.

In [ ]:
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
32/32 [==============================] - 3s 11ms/step - loss: 1.1174 - accuracy: 0.3399 - val_loss: 1.1140 - val_accuracy: 0.3318
Epoch 2/20
32/32 [==============================] - 0s 4ms/step - loss: 1.0937 - accuracy: 0.3656 - val_loss: 1.0986 - val_accuracy: 0.3134
Epoch 3/20
32/32 [==============================] - 0s 3ms/step - loss: 1.0871 - accuracy: 0.3538 - val_loss: 1.0946 - val_accuracy: 0.3410
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 1.0736 - accuracy: 0.4269 - val_loss: 1.0776 - val_accuracy: 0.3594
Epoch 5/20
32/32 [==============================] - 0s 3ms/step - loss: 1.0615 - accuracy: 0.4466 - val_loss: 1.0615 - val_accuracy: 0.4654
Epoch 6/20
32/32 [==============================] - 0s 4ms/step - loss: 1.0421 - accuracy: 0.4901 - val_loss: 1.0645 - val_accuracy: 0.3779
Epoch 7/20
32/32 [==============================] - 0s 3ms/step - loss: 1.0261 - accuracy: 0.5119 - val_loss: 1.0356 - val_accuracy: 0.4931
Epoch 8/20
32/32 [=

In [39]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet without regularization results,")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet without regularization results,
Acc	Train:0.62	Val:0.54	Test:0.48
loss	Train:0.84	Val:0.95	Test:0.92


## Part a: Impact of regularization layer

### L1 regularization

In [136]:
l1_val = 1e-6
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20,
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 4ms/step - loss: 1.1053 - accuracy: 0.3520 - val_loss: 1.0861 - val_accuracy: 0.3835
Epoch 2/20
65/65 [==============================] - 0s 3ms/step - loss: 1.0700 - accuracy: 0.4284 - val_loss: 1.0665 - val_accuracy: 0.3985
Epoch 3/20
65/65 [==============================] - 0s 2ms/step - loss: 1.0288 - accuracy: 0.4758 - val_loss: 1.0062 - val_accuracy: 0.5414
Epoch 4/20
65/65 [==============================] - 0s 2ms/step - loss: 0.9774 - accuracy: 0.5213 - val_loss: 0.9812 - val_accuracy: 0.5414
Epoch 5/20
65/65 [==============================] - 0s 2ms/step - loss: 0.9539 - accuracy: 0.5329 - val_loss: 0.9705 - val_accuracy: 0.5414
Epoch 6/20
65/65 [==============================] - 0s 3ms/step - loss: 0.9302 - accuracy: 0.5658 - val_loss: 0.9697 - val_accuracy: 0.5414
Epoch 7/20
65/65 [==============================] - 0s 3ms/step - loss: 0.9198 - accuracy: 0.5571 - val_loss: 0.9804 - val_accuracy: 0.5338
Epoch 8/20
65/65 [==

In [137]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with L1 regularization results, L1={l1_val}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with L1 regularization results, L1=1e-06
Acc	Train:0.67	Val:0.55	Test:0.52
loss	Train:0.78	Val:0.95	Test:0.94


In [45]:
l1_val = 0.1
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(l1_val)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 8ms/step - loss: 100.2238 - accuracy: 0.3395 - val_loss: 85.2452 - val_accuracy: 0.4286
Epoch 2/20
65/65 [==============================] - 0s 5ms/step - loss: 72.5081 - accuracy: 0.3424 - val_loss: 60.0493 - val_accuracy: 0.4586
Epoch 3/20
65/65 [==============================] - 0s 5ms/step - loss: 49.7062 - accuracy: 0.3511 - val_loss: 39.7303 - val_accuracy: 0.3684
Epoch 4/20
65/65 [==============================] - 0s 6ms/step - loss: 31.7356 - accuracy: 0.3356 - val_loss: 24.2108 - val_accuracy: 0.4060
Epoch 5/20
65/65 [==============================] - 0s 5ms/step - loss: 18.5604 - accuracy: 0.3385 - val_loss: 13.4950 - val_accuracy: 0.3684
Epoch 6/20
65/65 [==============================] - 0s 6ms/step - loss: 10.2154 - accuracy: 0.3424 - val_loss: 7.6195 - val_accuracy: 0.3233
Epoch 7/20
65/65 [==============================] - 0s 5ms/step - loss: 6.6071 - accuracy: 0.3230 - val_loss: 5.9653 - val_accuracy: 0.3308
Epoch 8/

In [47]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with L1 regularization results, L1={l1_val}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with L1 regularization results, L1=0.1
Acc	Train:0.34	Val:0.34	Test:0.34
loss	Train:1.32	Val:1.32	Test:1.32


### L2 regularization

In [62]:
l2_val = 1e-6
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 7ms/step - loss: 1.0916 - accuracy: 0.3975 - val_loss: 1.0631 - val_accuracy: 0.4361
Epoch 2/20
65/65 [==============================] - 0s 5ms/step - loss: 1.0270 - accuracy: 0.4971 - val_loss: 1.0387 - val_accuracy: 0.5263
Epoch 3/20
65/65 [==============================] - 0s 4ms/step - loss: 0.9898 - accuracy: 0.5068 - val_loss: 1.0133 - val_accuracy: 0.4812
Epoch 4/20
65/65 [==============================] - 0s 4ms/step - loss: 0.9418 - accuracy: 0.5609 - val_loss: 0.9764 - val_accuracy: 0.5639
Epoch 5/20
65/65 [==============================] - 0s 5ms/step - loss: 0.9096 - accuracy: 0.5957 - val_loss: 0.9687 - val_accuracy: 0.5865
Epoch 6/20
65/65 [==============================] - 0s 5ms/step - loss: 0.8982 - accuracy: 0.5822 - val_loss: 0.9661 - val_accuracy: 0.5639
Epoch 7/20
65/65 [==============================] - 0s 4ms/step - loss: 0.8720 - accuracy: 0.6044 - val_loss: 0.9642 - val_accuracy: 0.5865
Epoch 8/20
65/65 [==

In [63]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with L2 regularization results, L2={l2_val}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with L2 regularization results, L2=1e-06
Acc	Train:0.67	Val:0.59	Test:0.55
loss	Train:0.79	Val:0.95	Test:0.92


In [65]:
l2_val = 1e-1
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh', kernel_regularizer=tf.keras.regularizers.L2(l2_val)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 7ms/step - loss: 4.5832 - accuracy: 0.3191 - val_loss: 4.1805 - val_accuracy: 0.3459
Epoch 2/20
65/65 [==============================] - 0s 5ms/step - loss: 3.8448 - accuracy: 0.4101 - val_loss: 3.5323 - val_accuracy: 0.3534
Epoch 3/20
65/65 [==============================] - 0s 5ms/step - loss: 3.2676 - accuracy: 0.4942 - val_loss: 3.0491 - val_accuracy: 0.5113
Epoch 4/20
65/65 [==============================] - 0s 5ms/step - loss: 2.8274 - accuracy: 0.5164 - val_loss: 2.6676 - val_accuracy: 0.4586
Epoch 5/20
65/65 [==============================] - 0s 5ms/step - loss: 2.4892 - accuracy: 0.5329 - val_loss: 2.3636 - val_accuracy: 0.5263
Epoch 6/20
65/65 [==============================] - 0s 5ms/step - loss: 2.2223 - accuracy: 0.5445 - val_loss: 2.1439 - val_accuracy: 0.5338
Epoch 7/20
65/65 [==============================] - 0s 5ms/step - loss: 2.0189 - accuracy: 0.5542 - val_loss: 1.9717 - val_accuracy: 0.5188
Epoch 8/20
65/65 [==

In [66]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with L2 regularization results, L2={l2_val}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with L2 regularization results, L2=0.1
Acc	Train:0.63	Val:0.56	Test:0.57
loss	Train:1.19	Val:1.28	Test:1.24


### DropOut regularization

In [72]:
dropout_rate = 1e-4
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 7ms/step - loss: 1.1129 - accuracy: 0.3269 - val_loss: 1.0867 - val_accuracy: 0.3609
Epoch 2/20
65/65 [==============================] - 0s 4ms/step - loss: 1.0800 - accuracy: 0.3965 - val_loss: 1.0655 - val_accuracy: 0.4436
Epoch 3/20
65/65 [==============================] - 0s 4ms/step - loss: 1.0563 - accuracy: 0.4458 - val_loss: 1.0513 - val_accuracy: 0.4511
Epoch 4/20
65/65 [==============================] - 0s 4ms/step - loss: 1.0140 - accuracy: 0.5019 - val_loss: 1.0119 - val_accuracy: 0.5338
Epoch 5/20
65/65 [==============================] - 0s 5ms/step - loss: 0.9774 - accuracy: 0.5435 - val_loss: 1.0157 - val_accuracy: 0.4436
Epoch 6/20
65/65 [==============================] - 0s 5ms/step - loss: 0.9532 - accuracy: 0.5445 - val_loss: 1.0031 - val_accuracy: 0.4737
Epoch 7/20
65/65 [==============================] - 0s 4ms/step - loss: 0.9270 - accuracy: 0.5754 - val_loss: 0.9776 - val_accuracy: 0.5414
Epoch 8/20
65/65 [==

In [73]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with Drop out regularization results, dropout={dropout_rate}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with Drop out regularization results, dropout=0.0001
Acc	Train:0.66	Val:0.60	Test:0.59
loss	Train:0.78	Val:0.94	Test:0.95


In [15]:
dropout_rate = 0.7
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 2s 17ms/step - loss: 1.1993 - accuracy: 0.3240 - val_loss: 1.0931 - val_accuracy: 0.4361
Epoch 2/20
65/65 [==============================] - 1s 15ms/step - loss: 1.1949 - accuracy: 0.3395 - val_loss: 1.0993 - val_accuracy: 0.3459
Epoch 3/20
65/65 [==============================] - 1s 15ms/step - loss: 1.1463 - accuracy: 0.3453 - val_loss: 1.0871 - val_accuracy: 0.3609
Epoch 4/20
65/65 [==============================] - 1s 15ms/step - loss: 1.1430 - accuracy: 0.3685 - val_loss: 1.0798 - val_accuracy: 0.4135
Epoch 5/20
65/65 [==============================] - 1s 15ms/step - loss: 1.1340 - accuracy: 0.3617 - val_loss: 1.0756 - val_accuracy: 0.4662
Epoch 6/20
65/65 [==============================] - 1s 16ms/step - loss: 1.1187 - accuracy: 0.3675 - val_loss: 1.0729 - val_accuracy: 0.4286
Epoch 7/20
65/65 [==============================] - 1s 15ms/step - loss: 1.1162 - accuracy: 0.3752 - val_loss: 1.0638 - val_accuracy: 0.4737
Epoch 8/20
65

In [16]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"LeNet with Drop out regularization results, dropout={dropout_rate}")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
LeNet with Drop out regularization results, dropout=0.7
Acc	Train:0.56	Val:0.50	Test:0.52
loss	Train:0.95	Val:0.99	Test:0.99


## Part b: Impact of kernel count

In [125]:
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=1, kernel_size=5, activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=1, kernel_size=5, strides=(1, 1), activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=1, kernel_size=5, activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20, 
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 4ms/step - loss: 1.0988 - accuracy: 0.3366 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 2/20
65/65 [==============================] - 0s 2ms/step - loss: 1.0987 - accuracy: 0.3366 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 3/20
65/65 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3366 - val_loss: 1.0986 - val_accuracy: 0.3383


In [126]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"Question 1 part b results")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
Question 1 part b results
Acc	Train:0.34	Val:0.34	Test:0.34
loss	Train:1.10	Val:1.10	Test:1.10


## Part c: Impact of kernel size

In [127]:
lenet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=3, activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='tanh'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2,),
    tf.keras.layers.Conv2D(filters=120, kernel_size=2, activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(3, activation='softmax')
])

lenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)]
lenet.fit(xtrain32by32, ytrain32by32, validation_data=(xval32by32, yval32by32), epochs=20,
          batch_size=16, callbacks=callbacks)

Epoch 1/20
65/65 [==============================] - 1s 4ms/step - loss: 1.1175 - accuracy: 0.3424 - val_loss: 1.0880 - val_accuracy: 0.3759
Epoch 2/20
65/65 [==============================] - 0s 2ms/step - loss: 1.0726 - accuracy: 0.4226 - val_loss: 1.0558 - val_accuracy: 0.4286
Epoch 3/20
65/65 [==============================] - 0s 3ms/step - loss: 1.0275 - accuracy: 0.5000 - val_loss: 1.0249 - val_accuracy: 0.4962
Epoch 4/20
65/65 [==============================] - 0s 2ms/step - loss: 0.9973 - accuracy: 0.4778 - val_loss: 1.0001 - val_accuracy: 0.5263
Epoch 5/20
65/65 [==============================] - 0s 3ms/step - loss: 0.9754 - accuracy: 0.5145 - val_loss: 0.9923 - val_accuracy: 0.4812
Epoch 6/20
65/65 [==============================] - 0s 3ms/step - loss: 0.9455 - accuracy: 0.5493 - val_loss: 0.9789 - val_accuracy: 0.5714
Epoch 7/20
65/65 [==============================] - 0s 3ms/step - loss: 0.9336 - accuracy: 0.5358 - val_loss: 0.9707 - val_accuracy: 0.5263
Epoch 8/20
65/65 [==

In [128]:
hist = dict()
hist['loss'], hist['accuracy'] = lenet.evaluate(xtrain32by32, ytrain32by32, verbose=0)
hist['val_loss'], hist['val_accuracy'] = lenet.evaluate(xval32by32, yval32by32, verbose=0)
hist['test_loss'], hist['test_accuracy'] = lenet.evaluate(xtest32by32, ytest32by32, verbose=0)
print('=========== Report ===========')
print(f"Question 1 part b results")
print(f"Acc\tTrain:{hist['accuracy']:.2f}\tVal:{hist['val_accuracy']:.2f}\tTest:{hist['test_accuracy']:.2f}")
print(f"loss\tTrain:{hist['loss']:.2f}\tVal:{hist['val_loss']:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
Question 1 part b results
Acc	Train:0.62	Val:0.59	Test:0.52
loss	Train:0.84	Val:0.95	Test:0.97


# Question 4: Inception

In [ ]:
for i, fpercent in enumerate(np.arange(0, 1.1, 0.1)):
    inceptionv3 = InceptionV3(include_top=False)

    freezed_layers = inceptionv3.layers[:int(fpercent * len(inceptionv3.layers))]
    for l in freezed_layers:
        l.trainable = False

    inputs = tf.keras.layers.Input((500, 500, 3))
    pooling_layer = tf.keras.layers.GlobalMaxPooling2D(data_format="channels_last")
    dropout_layer = tf.keras.layers.Dropout(0.5)
    output_layer = tf.keras.layers.Dense(3, activation="softmax")

    x = inceptionv3(inputs)
    x = pooling_layer(x)
    outputs = output_layer(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(xtrain, ytrain, validation_data=(xval, yval), epochs=7, verbose=0)
    hist = history.history

    hist['test_loss'], hist['test_accuracy'] = model.evaluate(xtest, ytest, verbose=0)
    print('=========== Report ===========')
    print(f"Reporting results when {fpercent:.2f} of layers are frozen,")
    print(f"Acc\tTrain:{hist['accuracy'][-1]:.2f}\tVal:{hist['val_accuracy'][-1]:.2f}\tTest:{hist['test_accuracy']:.2f}")
    print(f"loss\tTrain:{hist['loss'][-1]:.2f}\tVal:{hist['val_loss'][-1]:.2f}\tTest:{hist['test_loss']:.2f}")

=========== Report ===========
Reporting results when 0.00 of layers freezed,
Acc	Train:1.00	Val:0.95	Test:0.90
loss	Train:0.00	Val:0.14	Test:0.32
=========== Report ===========
Reporting results when 0.10 of layers freezed,
Acc	Train:1.00	Val:0.80	Test:0.81
loss	Train:0.00	Val:0.71	Test:0.56
=========== Report ===========
Reporting results when 0.20 of layers freezed,
Acc	Train:1.00	Val:0.66	Test:0.67
loss	Train:0.02	Val:0.90	Test:0.86
=========== Report ===========
Reporting results when 0.30 of layers freezed,
Acc	Train:1.00	Val:0.62	Test:0.70
loss	Train:0.01	Val:1.28	Test:0.91
=========== Report ===========
Reporting results when 0.40 of layers freezed,
Acc	Train:1.00	Val:0.62	Test:0.65
loss	Train:0.01	Val:1.17	Test:1.20
=========== Report ===========
Reporting results when 0.50 of layers freezed,
Acc	Train:0.99	Val:0.65	Test:0.73
loss	Train:0.03	Val:0.99	Test:0.83
=========== Report ===========
Reporting results when 0.60 of layers freezed,
Acc	Train:0.99	Val:0.55	Test:0.62
loss	T